# 🚀 English-Vietnamese Translation Model
## Train on Google Colab

**Requirements:**
- Runtime → Change runtime type → **GPU** (T4 recommended)
- ~13 hours for 10 epochs with full data

**Features:**
- ✅ Auto-save checkpoint mỗi 15 phút
- ✅ Tự động resume từ checkpoint nếu bị ngắt
- ✅ Lưu vào Google Drive để không mất dữ liệu

---

In [ ]:
# 1. Check GPU
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ No GPU! Go to Runtime → Change runtime type → GPU")

In [ ]:
# 2. Mount Google Drive (để lưu checkpoint tự động)
from google.colab import drive
drive.mount('/content/drive')

# Tạo thư mục lưu checkpoint
import os
CHECKPOINT_DIR = '/content/drive/MyDrive/EV_Translation_Checkpoint'
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
print(f"✓ Checkpoint directory: {CHECKPOINT_DIR}")

In [ ]:
# 3. Clone repository
!rm -rf EV_Translate_Modle_NLP_Project
!git clone https://github.com/TranKien2005/EV_Translate_Modle_NLP_Project.git
%cd EV_Translate_Modle_NLP_Project

In [ ]:
# 4. Install dependencies (skip torch - already installed on Colab)
!pip install -q datasets sentencepiece sacrebleu google-generativeai python-dotenv tqdm tensorboard seaborn pyyaml huggingface_hub

In [ ]:
# 5. Create .env with your tokens
# REPLACE THESE WITH YOUR ACTUAL TOKENS!
HF_TOKEN = "YOUR_HUGGINGFACE_TOKEN"  # Get from https://huggingface.co/settings/tokens
GEMINI_API_KEY = "YOUR_GEMINI_API_KEY"  # Optional, for evaluation

with open('.env', 'w') as f:
    f.write(f'HF_TOKEN={HF_TOKEN}\n')
    f.write(f'GEMINI_API_KEY={GEMINI_API_KEY}\n')

print('✓ .env created')
print('⚠️ Make sure to replace the placeholder tokens above!')

In [ ]:
# 6. Download PhoMT dataset (~500MB)
!python scripts/download_phomt.py

In [ ]:
# 7. Preprocess data (may take 10-15 minutes for full dataset)
# Use --max-samples 100000 for quick test
!python scripts/preprocess_data.py

In [ ]:
# 8. Configure for Colab training
import yaml

with open('config/config.yaml', 'r') as f:
    cfg = yaml.safe_load(f)

# Training settings
cfg['data']['source'] = 'processed'
cfg['training']['epochs'] = 10
cfg['training']['batch_size'] = 128  # T4 has 16GB, can handle larger batch
cfg['hardware']['num_workers'] = 2  # Colab works better with fewer workers

with open('config/config.yaml', 'w') as f:
    yaml.dump(cfg, f, default_flow_style=False)

print('✓ Config updated for Colab')
print(f"  Epochs: {cfg['training']['epochs']}")
print(f"  Batch size: {cfg['training']['batch_size']}")

## 🔄 Resume Training Setup

Hệ thống sẽ:
1. Kiểm tra checkpoint trong Google Drive
2. Nếu có → resume training từ checkpoint đó
3. Nếu không → train từ đầu
4. **Auto-save mỗi 15 phút** vào Google Drive

In [ ]:
# 9. Kiểm tra checkpoint hiện có trong Drive
import os
import shutil

CHECKPOINT_FILE = os.path.join(CHECKPOINT_DIR, 'current_checkpoint.pt')
TOKENIZER_DIR = os.path.join(CHECKPOINT_DIR, 'tokenizers')

# Kiểm tra có checkpoint không
RESUME_FROM = None
if os.path.exists(CHECKPOINT_FILE):
    print(f"✅ Found existing checkpoint: {CHECKPOINT_FILE}")
    print(f"   Size: {os.path.getsize(CHECKPOINT_FILE) / 1024 / 1024:.1f} MB")
    
    # Copy checkpoint về local
    shutil.copy(CHECKPOINT_FILE, 'checkpoints/resume_checkpoint.pt')
    RESUME_FROM = 'checkpoints/resume_checkpoint.pt'
    print(f"   → Copied to local for resume")
    
    # Copy tokenizers nếu có
    if os.path.exists(TOKENIZER_DIR):
        os.makedirs('checkpoints/tokenizers', exist_ok=True)
        for f in os.listdir(TOKENIZER_DIR):
            shutil.copy(os.path.join(TOKENIZER_DIR, f), f'checkpoints/tokenizers/{f}')
        print(f"   → Copied tokenizers from Drive")
else:
    print("📌 No checkpoint found in Drive. Will train from scratch.")

In [ ]:
# 10. Custom Trainer với Auto-Save mỗi 15 phút
import sys
sys.path.insert(0, '.')

import time
import shutil
import threading
from src.train import Trainer
from src.utils import save_checkpoint

class AutoSaveTrainer(Trainer):
    """Trainer với tính năng auto-save mỗi 15 phút."""
    
    def __init__(self, save_dir, save_interval=900, **kwargs):
        """
        Args:
            save_dir: Thư mục lưu checkpoint (Google Drive)
            save_interval: Khoảng thời gian giữa các lần lưu (giây), mặc định 900s = 15 phút
        """
        super().__init__(**kwargs)
        self.save_dir = save_dir
        self.save_interval = save_interval
        self.last_save_time = time.time()
        self._stop_auto_save = False
        
    def _auto_save_checkpoint(self):
        """Lưu checkpoint hiện tại vào Drive."""
        if self.model is None:
            return
            
        try:
            # Lưu vào local trước
            local_path = 'checkpoints/auto_save_temp.pt'
            save_checkpoint(
                self.model,
                self.optimizer,
                self.current_epoch,
                self.best_val_loss if hasattr(self, 'best_val_loss') else float('inf'),
                local_path,
                best_val_loss=self.best_val_loss if hasattr(self, 'best_val_loss') else float('inf')
            )
            
            # Copy sang Drive (ghi đè)
            drive_path = os.path.join(self.save_dir, 'current_checkpoint.pt')
            shutil.copy(local_path, drive_path)
            
            # Lưu tokenizers
            tokenizer_dir = os.path.join(self.save_dir, 'tokenizers')
            os.makedirs(tokenizer_dir, exist_ok=True)
            if os.path.exists('checkpoints/tokenizers'):
                for f in os.listdir('checkpoints/tokenizers'):
                    shutil.copy(f'checkpoints/tokenizers/{f}', os.path.join(tokenizer_dir, f))
            
            self.last_save_time = time.time()
            print(f"\n💾 Auto-saved checkpoint to Drive at epoch {self.current_epoch}")
            
        except Exception as e:
            print(f"\n⚠️ Auto-save failed: {e}")
    
    def train_epoch(self):
        """Override để thêm logic auto-save."""
        result = super().train_epoch()
        
        # Kiểm tra có cần auto-save không
        elapsed = time.time() - self.last_save_time
        if elapsed >= self.save_interval:
            self._auto_save_checkpoint()
        
        return result

print('✓ AutoSaveTrainer defined')
print(f'  Auto-save interval: {900}s (15 minutes)')
print(f'  Save directory: {CHECKPOINT_DIR}')

In [ ]:
# 11. Start Training với Auto-Save!
# Checkpoint sẽ được lưu vào Google Drive mỗi 15 phút
# Nếu bị ngắt, chạy lại từ cell 1 và nó sẽ tự resume

trainer = AutoSaveTrainer(
    save_dir=CHECKPOINT_DIR,
    save_interval=900  # 15 phút
)
trainer.setup()

if RESUME_FROM:
    print(f"\n🔄 Resuming training from: {RESUME_FROM}")
    trainer.train(resume_from=RESUME_FROM)
else:
    print("\n🚀 Starting training from scratch")
    trainer.train()

# Lưu checkpoint cuối cùng
print("\n" + "="*60)
print("✅ Training Complete!")
print("="*60)

# Lưu best model vào Drive
if os.path.exists('checkpoints/best_model.pt'):
    shutil.copy('checkpoints/best_model.pt', os.path.join(CHECKPOINT_DIR, 'best_model.pt'))
    print(f"✓ Best model saved to: {CHECKPOINT_DIR}/best_model.pt")

In [ ]:
# 12. Test translations
from src.evaluate import load_translator

translator = load_translator()

test_sentences = [
    "Hello, how are you?",
    "Thank you very much.",
    "I love learning new languages.",
    "The weather is beautiful today.",
    "What is your name?"
]

print("\n" + "="*60)
print("Translation Results")
print("="*60)
for sentence in test_sentences:
    translation = translator.translate(sentence)
    print(f"EN: {sentence}")
    print(f"VI: {translation}")
    print("-"*40)

In [ ]:
# 13. Evaluate BLEU score
!python -m src.evaluate --checkpoint checkpoints/best_model.pt --split test --max-samples 1000

In [ ]:
# 14. Save final model to Google Drive
import shutil

final_save_path = '/content/drive/MyDrive/EV_Translation_Model_Final'
os.makedirs(final_save_path, exist_ok=True)

# Copy all model files
shutil.copy('checkpoints/best_model.pt', final_save_path)
shutil.copytree('checkpoints/tokenizers', f'{final_save_path}/tokenizers', dirs_exist_ok=True)
shutil.copy('config/config.yaml', final_save_path)

print(f"✓ Final model saved to {final_save_path}")
print("\nFiles saved:")
for f in os.listdir(final_save_path):
    print(f"  - {f}")

---

## 📋 Cách Resume Training

Nếu Colab bị ngắt, chỉ cần **chạy lại từ cell đầu tiên**. Hệ thống sẽ:
1. Mount lại Google Drive
2. Tự động tìm checkpoint trong Drive
3. Resume training từ checkpoint đó

**Không cần làm gì thêm!** 🎉

---

## 💡 Tips

1. **Colab Pro** gives longer runtime and better GPUs (A100)
2. Checkpoint được lưu vào Drive mỗi 15 phút
3. Nếu muốn reset và train lại từ đầu, xóa folder `EV_Translation_Checkpoint` trong Drive